# 서울특별시 공공자전거 따릉이 대여이력 연도별 전처리
* 데이터 출처 : [데이터셋> 데이터 이용하기 | 서울열린데이터광장](http://data.seoul.go.kr/dataList/datasetView.do?infId=OA-15182&srvType=F&serviceKind=1&currentPageNo=1)
* 전체 데이터를 가져와서 연도별로 병합(concat)을 해줍니다.

In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
%ls data/

In [ ]:
def preprocessing(df):
    # 컬럼에 있는 '를 제거한다.
    df.columns = df.columns.str.strip("'")
    # 불필요하게 '가 들어가 있다. 스트링 타입으로 되어있는 데이터에서 '을 제거하자.
    df = df.apply(lambda x: x.str.strip("'") if x.dtype == np.dtype('object') else x)
    return df

In [3]:
# 컬럼명이 2017년을 기점으로 달라지기 때문에 최근 컬럼에 맞게 컬럼명을 변경해 준다.
# 우선 최근 컬럼명을 가져온다.
df_for_columns = pd.read_csv("data/서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv", 
                             encoding="cp949")
df_for_columns.columns = df_for_columns.columns.str.strip("'")
cols = df_for_columns.columns
len(cols)

11

In [5]:
df_for_columns

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,'SPB-11906','2018-06-16 00:00:07','932',' 예일여중',1,'2018-06-16 00:16:41','934',' 신사동 성당',11,15,2160
1,'SPB-16954','2018-06-16 00:00:07','272',' 당산육갑문',16,'2018-06-16 02:00:36','1827',' 독산보도육교 앞 자전거 보관소',9,120,18100
2,'SPB-09676','2018-06-16 00:00:11','1036',' 고덕동 주양쇼핑',7,'2018-06-16 00:24:05','1038',' 강일 다솜 어린이 공원',9,23,3350
3,'SPB-07143','2018-06-16 00:00:15','1726',' 삼익세라믹아파트교차로',7,'2018-06-16 00:15:31','1706',' 기업은행 앞',9,14,1480
4,'SPB-13007','2018-06-16 00:00:16','2058',' 노량진동 맥도널드앞',10,'2018-06-16 00:14:25','258',' 신길역3번출구',8,13,2920
...,...,...,...,...,...,...,...,...,...,...,...
586299,'SPB-15985','2018-06-30 23:59:55','505',' 자양사거리 광진아크로텔 앞',9,'2018-07-01 00:09:38','593','자양중앙나들목',5,9,1230
586300,'SPB-15081','2018-06-30 23:59:55','1328',' 월곡역 3번출구',7,'2018-07-01 00:18:02','647',' 신이문역 1번출구',7,17,3600
586301,'SPB-13677','2018-06-30 23:59:57','2376',' 수서역 6번출구 앞',2,'2018-07-01 00:20:38','2387',' 래미안강남힐즈 사거리',5,20,2590
586302,'SPB-06577','2018-06-30 23:59:57','1251',' 석촌역 2번출구',2,'2018-07-01 00:57:29','1245',' 문정 법조단지9',4,57,6640


In [ ]:
df = pd.read_excel('data/서울특별시 공공자전거 대여정보_201807_01.xlsx', encoding='cp949')
df.head()

In [ ]:
def get_file_list_by_year(year):
    file_list = []
    for root, dirs, files in os.walk("data/"):
        for filename in files:
            if filename.startswith(f'서울특별시 공공자전거 대여이력 정보_{year}년'):
                file_list.append(filename)
            elif filename.startswith(f'서울특별시 공공자전거 대여정보_{year}'):
                file_list.append(filename)
    return sorted(file_list)

get_file_list_by_year(2019)

In [ ]:
def concat_data(year):
    df_lists = []
    # 파일별로 데이터를 가져온다.
    files = get_file_list_by_year(year)
    for filename in sorted(files):
        # 파일을 읽어온다. csv와 xlsx 파일을 구분해 불러온다.
        df_temp = read_file(filename, year)
        print(df_temp)
        # 해당 데이터프레임이 비어있지 않다면 병합하기 위해 리스트에 담는다.
        if not df_temp.empty :
            print(filename, df_temp.shape)
            df_lists.append(df_temp)
    # 리스트가 비어있지 않다면 데이터프레임을 합쳐준다.       
    if df_lists :    
        df = pd.concat(df_lists)
    return df        

In [ ]:
def read_file(filename, year):
    folder_path = 'data/'
    if filename.startswith(f'서울특별시 공공자전거 대여이력 정보_{year}년'):
        # 서울시 정보소통광장에서 다운로드 받은 데이터의 인코딩이 cp949이기 때문에 인코딩을 지정해 주어야 한다.
        df_temp = pd.read_csv(folder_path+filename, encoding='cp949', low_memory=False)
        # 데이터에 불필요한 따옴표(')가 들어가 있다. 제거해 준다.
        df_temp = preprocessing(df_temp)
        # 2017년 중간에 컬럼명이 변경되었다. 2018년 이전 데이터에 대해서 최근 컬럼으로 컬럼명을 변경해 준다. 
        # 최근 컬럼명 cols는 위 셀에서 구한 것을 사용한다.
        if year < 2018 :
            df_temp.columns = cols
    elif filename.startswith(f'서울특별시 공공자전거 대여정보_{year}') and filename.endswith(f'xlsx'):
        df_temp = pd.read_excel(folder_path+filename, encoding='cp949')
        df_temp.columns = cols
        
    elif filename.startswith(f'서울특별시 공공자전거 대여정보_{year}') and filename.endswith(f'csv'):
        df_temp = pd.read_csv(folder_path+filename, encoding='cp949')
        df_temp.columns = cols
    else:
        df_temp = pd.DataFrame([])
    return  df_temp

In [ ]:
%time df_2016 = concat_data(2016)
print(df_2016.shape)

In [ ]:
%time df_2017 = concat_data(2017)
print(df_2017.shape)

In [ ]:
%time df_2018 = concat_data(2018)
print(df_2018.shape)

In [ ]:
filename = "서울특별시 공공자전거 대여정보_201905.csv"
year = 2019
read_file(filename, year).head()

In [ ]:
filename.startswith(f'서울특별시 공공자전거 대여정보_{year}')
# filename.endswith(f'csv')

In [ ]:
%time df_2019 = concat_data(2019)
print(df_2019.shape)

In [ ]:
df_2019.head()

In [ ]:
df_2019["자전거번호SPB"] = df_2019["자전거번호"].apply(lambda x: x.split("-")[0])
df_2019["자전거번호No"] = df_2019["자전거번호"].apply(lambda x: x.split("-")[1])

In [ ]:
df_2019["자전거번호SPB"].unique()

In [ ]:
df_2019["대여대여소명"].isnull().sum()

In [ ]:
df_2019["반납대여소명"].isnull().sum()

In [ ]:
df_2016.head()

In [ ]:
df_2017.head()

In [ ]:
df_2018.head()

In [ ]:
years = [2016, 2017, 2018, 2019]
for year in years:
    df = eval('df_'+str(year))
    df.to_csv('data/seoul-bike-{}.csv'.format(year), index=False)   
    print(df.shape)

In [ ]:
# 파일이 제대로 생성되었는지 확인한다.
%ls data/seoul-bike*

In [ ]:
# 생성된 파일 안에 내용이 제대로 들어가 있는지 확인한다.
df_2018 = pd.read_csv('data/seoul-bike-2018.csv', low_memory=False)
print(df_2018.shape)
df_2018.head()